In [31]:
import time
import tqdm #tqdm은 스테이터스 바 (진행 바)
from tqdm import tqdm_notebook
import pandas as pd
import numpy as np
from glob import glob
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import urllib.request
import time
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
# 크롬 드라이버 자동 업데이트
from webdriver_manager.chrome import ChromeDriverManager
# 브라우저 꺼짐 방지
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
# 불필요한 에러 메시지 없애기
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
service = Service(executable_path=ChromeDriverManager().install())

In [32]:
import warnings
warnings.filterwarnings(action = "ignore")

In [33]:
#웹 브라우저 열기
driver = webdriver.Chrome(options=chrome_options)
url = "https://tmacs.kotsa.or.kr/web/TG/TG200/TG2100S/Tg2102.jsp?mid=S1177"
driver.get(url)

In [53]:
#지자체(구별) 크롤링 엑셀 다운로드
jijace_list_raw = driver.find_element(By.ID, "jijace") #지자체 
jijace_list = jijace_list_raw.find_elements(By.TAG_NAME, "option") #지자체 옵션
jijace_names = [option.text for option in jijace_list] #구 이름 저장
jijace_names = jijace_names[1:]

search = driver.find_element(By.CSS_SELECTOR, "#content > div.cont_box > div.filterForm > div.btn_wrap.btn-2 > a.btn.btn_primary")
excel_btn = driver.find_element(By.CSS_SELECTOR, "#content > div.cont_box > div:nth-child(2) > div.btn_box.type02 > a.btn.exbtn")

for gu in tqdm_notebook(jijace_names):
    jijace_list_raw.send_keys(gu)
    time.sleep(3)
    search.click()
    time.sleep(2)
    excel_btn.click()
    time.sleep(2)

  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
acc2_files = glob("원본데이터/서울_사고유형2/교통사고 지표 현황*xls")

In [7]:
tmp_raw = []
for name in acc2_files:
    tmp = pd.read_excel(name, header = 1,usecols=[0, 1, 2, 3, 4, 8])
    tmp_raw.append(tmp)

seoul_acc2_raw = pd.concat(tmp_raw)

In [8]:
seoul_acc2_raw.rename(columns={'지자체': '구'}, inplace=True)

In [10]:
seoul_acc2_raw.dropna(inplace=True) #결측치 삭제

In [11]:
seoul_acc2_raw['연도'] = seoul_acc2_raw['연도'].astype(str).str.split('.').str[0]

In [13]:
# 피봇 테이블 직전 버전
seoul_acc2_raw.to_excel("./정제데이터/서울/서울_사고유형2_raw.xlsx") 

In [14]:
seoul_acc2 = seoul_acc2_raw.pivot_table( index = ["시도", "구", "연도"], columns = "사고유형2", values=['발생건수', '사망자수'], aggfunc = np.sum)
seoul_acc2

발생건수                                                        \
사고유형2       공작물추돌 길가장자리구역통행중 도로외이탈(기타) 도로외이탈(추락) 보도통행중    전도   전복  정면충돌   
시도 구   연도                                                                 
서울 강남구 2018  18.0      112.0       1.0       NaN  43.0   3.0  NaN  51.0   
       2019  13.0       96.0       NaN       NaN  66.0   2.0  NaN  76.0   
       2020   8.0       94.0       NaN       1.0  65.0   9.0  NaN  77.0   
       2021  11.0       70.0       1.0       NaN  58.0   7.0  1.0  68.0   
       2022   8.0       70.0       NaN       NaN  61.0   3.0  1.0  58.0   
...           ...        ...       ...       ...   ...   ...  ...   ...   
   중랑구 2018  11.0       33.0       NaN       NaN  22.0   8.0  1.0  38.0   
       2019  19.0       32.0       NaN       NaN  28.0   8.0  NaN  42.0   
       2020  17.0       29.0       NaN       NaN  13.0  10.0  1.0  45.0   
       2021   4.0       15.0       NaN       NaN  26.0   8.0  NaN  54.0   
       2022   6.0       27.0       NaN       2.0  25.0   5.0  NaN  47.0   

                            ... 사망자수                                      \
사고유형2       주차차량추돌 차대사람-기타  ... 정면충돌 주차차량추돌 차대사람-기타 차대차-기타 차도통행중 차량단독-기타   
시도 구   연도                   ...                                            
서울 강남구 2018    NaN   327.0  ...  2.0    NaN     2.0    4.0   1.0     1.0   
       2019    NaN   401.0  ...  1.0    NaN     3.0    3.0   2.0     0.0   
       2020    NaN   376.0  ...  1.0    NaN     1.0    3.0   2.0     2.0   
       2021    NaN   358.0  ...  0.0    NaN     4.0    1.0   0.0     0.0   
       2022    NaN   448.0  ...  0.0    NaN     3.0    5.0   0.0     0.0   
...            ...     ...  ...  ...    ...     ...    ...   ...     ...   
   중랑구 2018    NaN   163.0  ...  1.0    NaN     1.0    0.0   3.0     0.0   
       2019    NaN   166.0  ...  2.0    NaN     0.0    1.0   1.0     0.0   
       2020    NaN   180.0  ...  0.0    NaN     2.0    0.0   0.0     0.0   
       2021    NaN   142.0  ...  0.0    NaN     2.0    0.0   1.0     0.0   
       2022    1.0   166.0  ...  0.0    0.0     2.0    1.0   0.0     0.0   

                                       
사고유형2       추돌(진행중) 측면직각충돌  횡단중 후진중추돌  
시도 구   연도                              
서울 강남구 2018     2.0    2.0  4.0   0.0  
       2019     1.0    1.0  1.0   0.0  
       2020     1.0    2.0  4.0   0.0  
       2021     1.0    0.0  2.0   0.0  
       2022     1.0    3.0  4.0   0.0  
...             ...    ...  ...   ...  
   중랑구 2018     0.0    1.0  3.0   0.0  
       2019     0.0    1.0  6.0   0.0  
       2020     0.0    0.0  2.0   0.0  
       2021     0.0    0.0  3.0   0.0  
       2022     1.0    1.0  4.0   0.0  

[125 rows x 34 columns]

In [17]:
new_columns = []

for num in range(int(len(seoul_acc2.columns)/2)):
    new_columns.append('발생_'+seoul_acc2.columns[num][1])

for num in range(int(len(seoul_acc2.columns)/2)):
    new_columns.append('사망_'+seoul_acc2.columns[num][1])

seoul_acc2.columns = new_columns

seoul_acc2.reset_index(inplace=True)

In [19]:
seoul_acc2.to_excel("./정제데이터/서울/서울_사고유형2.xlsx") 

### 데이터 정제 -자동화

In [54]:
#지역 변수 선언
region_list = ["서울", "부산", "대구", "인천", "광주", "대전", "울산", "세종", "경기", "강원", "충북", "충남", "전북", "전남", "경북", "경남", "제주" ]
detail = "사고유형2"

for region in region_list:
    acc2_files = glob(f"원본데이터/{detail}/{region}_{detail}/교통사고 지표 현황*xls")
    tmp_raw = []

    for name in acc2_files:
        tmp = pd.read_excel(name, header = 1,usecols=[0, 1, 2, 3, 4, 8])
        tmp_raw.append(tmp)

    acc2_raw = pd.concat(tmp_raw)

    acc2_raw.rename(columns={'지자체': '구'}, inplace=True)

    acc2_raw = acc2_raw.dropna() #결측치 삭제

    acc2_raw['연도'] = acc2_raw['연도'].astype(str).str.split('.').str[0] #연도 str화

    # 피봇 테이블 직전 버전
    acc2_raw.to_excel(f"./정제데이터/{region}/{region}_{detail}_raw.xlsx") 

    acc2 = acc2_raw.pivot_table( index = ["시도", "구", "연도"], columns = f"{detail}", values=['발생건수', '사망자수'], aggfunc = np.sum)
    acc2

    #평탄화 작업
    new_columns = []

    for num in range(int(len(acc2.columns)/2)):
        new_columns.append('발생_'+acc2.columns[num][1])

    for num in range(int(len(acc2.columns)/2)):
        new_columns.append('사망_'+acc2.columns[num][1])

    acc2.columns = new_columns

    acc2.reset_index(inplace=True)

    # 최종 버전
    acc2.to_excel(f"./정제데이터/{region}/{region}_{detail}.xlsx") 